# Maximal Clique Problem

## Graph Generation

It was provided the functions below for graph generation and the solution to solve the Maximal Clique Problem:

In [49]:
import networkx as nx
import random

def create_graph(num_vertices, probabilidade_conexao, nome_arquivo):

    # Crie um grafo aleatório densamente conectado
    grafo = nx.fast_gnp_random_graph(num_vertices, probabilidade_conexao)

    # Abra o arquivo para escrita
    with open(nome_arquivo, 'w') as arquivo:
        # Escreva a quantidade de vértices e número de arestas na primeira linha
        arquivo.write(f"{num_vertices} {grafo.number_of_edges()}\n")

        # Escreva as arestas no formato de lista de adjacência
        for aresta in grafo.edges():
            arquivo.write(f"{aresta[0]+1} {aresta[1]+1}\n")  # +1 para ajustar os índices (começando em 1)

    print(f"Grafo densamente conectado gerado e salvo em '{nome_arquivo}'.")

In [50]:
def max_clique(nome_arquivo):
    # Abrir o arquivo e pular a primeira linha
    with open(nome_arquivo, 'r') as arquivo:
        next(arquivo)  # Pula a primeira linha

        # Lê o grafo a partir das linhas restantes
        G = nx.parse_adjlist(arquivo)

    # Encontrar todas as cliques maximais
    cliques_maximais = list(nx.find_cliques(G))

    # Determinar o tamanho da maior clique
    tamanho_maximo = max(len(clique) for clique in cliques_maximais)

    # Encontrar uma clique que tem o tamanho máximo
    clique_maxima = next(clique for clique in cliques_maximais if len(clique) == tamanho_maximo)

    # Imprimir apenas o tamanho e uma clique exemplo
    print("Tamanho da maior clique:", tamanho_maximo)
    print("Exemplo de clique máxima:", clique_maxima)



In [56]:
lista_vertices = [5,25,50,125,100,150]

for i in lista_vertices:
    create_graph(i, 0.7, f"grafos/grafo_{i}_vertices.txt")

Grafo densamente conectado gerado e salvo em 'grafos/grafo_5_vertices.txt'.
Grafo densamente conectado gerado e salvo em 'grafos/grafo_25_vertices.txt'.
Grafo densamente conectado gerado e salvo em 'grafos/grafo_50_vertices.txt'.
Grafo densamente conectado gerado e salvo em 'grafos/grafo_125_vertices.txt'.
Grafo densamente conectado gerado e salvo em 'grafos/grafo_100_vertices.txt'.
Grafo densamente conectado gerado e salvo em 'grafos/grafo_150_vertices.txt'.


## Algorithm Overview

## Validation

### Resultado Python

In [57]:
for i in lista_vertices:
    print("Arquivo com", i, "vertices:")
    max_clique(f"grafos/grafo_{i}_vertices.txt")

Arquivo com 5 vertices:
Tamanho da maior clique: 4
Exemplo de clique máxima: ['4', '5', '3', '2']
Arquivo com 25 vertices:
Tamanho da maior clique: 8
Exemplo de clique máxima: ['23', '8', '21', '16', '10', '19', '7', '2']
Arquivo com 50 vertices:
Tamanho da maior clique: 10
Exemplo de clique máxima: ['6', '27', '5', '23', '8', '35', '18', '33', '31', '3']
Arquivo com 125 vertices:
Tamanho da maior clique: 16
Exemplo de clique máxima: ['8', '89', '19', '76', '115', '9', '29', '92', '38', '119', '64', '70', '108', '17', '100', '103']
Arquivo com 100 vertices:
Tamanho da maior clique: 14
Exemplo de clique máxima: ['36', '55', '40', '4', '28', '68', '38', '67', '57', '73', '5', '95', '29', '59']
Arquivo com 150 vertices:
Tamanho da maior clique: 16
Exemplo de clique máxima: ['70', '20', '35', '57', '136', '117', '74', '13', '45', '47', '95', '90', '149', '26', '46', '42']


## Results

## Conclusion